In [4]:
import numpy as np
import pandas as pd
from scipy.special import expit  # sigmoid function

# List of file names
files = ['data1.csv', 'data12.csv']

# Loop through each file
for i, files in enumerate(files, start = 1):
    print(f"Processing file {i}: {files}")
    
    df = pd.read_csv(files)  # Replace 'your_file.csv' with the actual file path
    info = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # X refers to features  y target values
    X = info.iloc[:, :-1].values
    y = info.iloc[:, -1].values

    # Add a column of ones to the features for the bias term
    X = np.c_[np.ones(X.shape[0]), X]

    # Initialize parameters
    coefficient = np.zeros(X.shape[1])
    lr = 0.001
    lambda_parameter = 0.01
    num_of_iterations = 1000

    # Logistic sigmoid function
    def sigmoid(z):
        return 1 / (1 + np.exp(-z))

    # Regularized log loss function
    def log_loss(coefficient, X, y, lambda_parameter):
        m = len(y)
        h = sigmoid(np.dot(X, coefficient))
        h = np.clip(h, 1e-15, 1 - 1e-15)
        loss = -np.sum(y * np.log(h) + (1 - y) * np.log(1 - h)) / m
        reg_term = (lambda_parameter) * np.sum(coefficient[1:]**2)  # Exclude bias term
        return loss + reg_term

    # Gradient of regularized log loss function
    def gradient_ridge(coefficient, X, y, lambda_parameter):
        m = len(y)
        h = sigmoid(np.dot(X, coefficient)) # h= prediction, errors = h-y
        h = np.clip(h, 1e-15, 1 - 1e-15)
        gradient = np.dot(X.T, (h - y)) / m
        reg_term = 2 * (lambda_parameter) * np.r_[0, coefficient[1:]]  # Include 0 for bias term
        return gradient + reg_term 

    # Gradient descent for regularized log loss
    def gradient_descent(X, y, coefficient, lr, lambda_parameter, num_of_iterations):
        for itr in range(num_of_iterations):
            gradient = gradient_ridge(coefficient, X, y, lambda_parameter)
            coefficient -= lr * gradient
            if itr % 100 == 0:
                loss = log_loss(coefficient, X, y, lambda_parameter)
                print(f"Iterations {itr}, Loss: {loss}")
        return coefficient, loss
    
    def accuracy(X, coefficient):
        prob = sigmoid( np.dot(X, coefficient))
        return (prob >= 0.5).astype(int)

    # Apply gradient descent
    optimal_coefficient, loss = gradient_descent(X, y, coefficient, lr, lambda_parameter, num_of_iterations)
    

    # Display the optimal parameters
    print(f"Optimal Parameters for file {i}:")
    print(optimal_coefficient)
    print(loss)
    predict = accuracy(X, optimal_coefficient )
    accur = np.mean(predict == y) * 100
    print("accuracy", accur)

Processing file 1: data1.csv
Iterations 0, Loss: 0.6921578814351956
Iterations 100, Loss: 0.6036875220827
Iterations 200, Loss: 0.5328792593561658
Iterations 300, Loss: 0.4757958298479238
Iterations 400, Loss: 0.4293254288542837
Iterations 500, Loss: 0.39108166475527806
Iterations 600, Loss: 0.35925860670069737
Iterations 700, Loss: 0.33249344569447037
Iterations 800, Loss: 0.30975420466501935
Iterations 900, Loss: 0.2902538888858622
Optimal Parameters for file 1:
[-0.00167865  0.62533039]
0.2902538888858622
accuracy 99.4
Processing file 2: data12.csv
Iterations 0, Loss: 0.6566374516953546
Iterations 100, Loss: 0.45326291531055024
Iterations 200, Loss: 0.43167547786865235
Iterations 300, Loss: 0.4211472314078763
Iterations 400, Loss: 0.41469246778214774
Iterations 500, Loss: 0.4102466396022887
Iterations 600, Loss: 0.4069559235034316
Iterations 700, Loss: 0.4044018176202778
Iterations 800, Loss: 0.4023538994011773
Iterations 900, Loss: 0.40067351132005397
Optimal Parameters for file 2: